In [2]:
import numpy as np
from scipy.linalg import null_space

# Define the vertices, edges, and faces of the tetrahedron
vertices = [(0,), (1,), (2,), (3,)]  # 0-simplices
edges = [(0, 1), (0, 2), (0, 3), (1, 2), (1, 3), (2, 3)]  # 1-simplices
faces = [(0, 1, 2), (0, 1, 3), (0, 2, 3), (1, 2, 3)]  # 2-simplices

# Adjacency matrix for the tetrahedron
adj_matrix_tetrahedron = np.zeros((4, 4), dtype=int)
for edge in edges:
    i, j = edge
    adj_matrix_tetrahedron[i, j] = 1
    adj_matrix_tetrahedron[j, i] = 1  # Since the graph is undirected

# Define boundary operators based on the simplices
def boundary_operator_1(edges, vertices):
    boundary_matrix = np.zeros((len(vertices), len(edges)))
    for i, edge in enumerate(edges):
        boundary_matrix[edge[0], i] = -1
        boundary_matrix[edge[1], i] = 1
    return boundary_matrix

def boundary_operator_2(faces, edges):
    boundary_matrix = np.zeros((len(edges), len(faces)))
    for i, face in enumerate(faces):
        for j, edge in enumerate(edges):
            if set(edge).issubset(face):
                if face.index(edge[1]) == (face.index(edge[0]) + 1) % 3:
                    boundary_matrix[j, i] = 1
                else:
                    boundary_matrix[j, i] = -1
    return boundary_matrix

# Construct the boundary matrices
boundary_1 = boundary_operator_1(edges, vertices)
boundary_2 = boundary_operator_2(faces, edges)

# Calculate the cohomology groups using the null space of the coboundary matrices
def calculate_cohomology_groups(boundary_1, boundary_2):
    # The coboundary matrices are the transposes of the boundary matrices
    coboundary_1 = boundary_1.T
    coboundary_2 = boundary_2.T
    
    # The kernel of the coboundary matrix gives us the cohomology group
    H_cohomology_0 = null_space(coboundary_1).shape[1]
    H_cohomology_1 = null_space(coboundary_2).shape[1] - np.linalg.matrix_rank(boundary_1)
    # For H_cohomology_2, since there are no 3-simplices, it's just the number of 2-simplices minus the rank of boundary_2
    H_cohomology_2 = len(faces) - np.linalg.matrix_rank(boundary_2)
    
    return H_cohomology_0, H_cohomology_1, H_cohomology_2

# Now we can compute the cohomology groups for the tetrahedron
H_cohomology_groups = calculate_cohomology_groups(boundary_1, boundary_2)

# Output the adjacency matrix and the cohomology groups
print("Adjacency Matrix for the Tetrahedron:\n", adj_matrix_tetrahedron)
print("Cohomology Groups:\n", "H^0:", H_cohomology_groups[0], 
      ", H^1:", H_cohomology_groups[1], ", H^2:", H_cohomology_groups[2])

Adjacency Matrix for the Tetrahedron:
 [[0 1 1 1]
 [1 0 1 1]
 [1 1 0 1]
 [1 1 1 0]]
Cohomology Groups:
 H^0: 1 , H^1: 0 , H^2: 1


In [5]:
import numpy as np
from scipy.linalg import null_space
from itertools import combinations

def boundary_operator_1(edges, vertices):
    boundary_matrix = np.zeros((len(vertices), len(edges)))
    for i, edge in enumerate(edges):
        boundary_matrix[edge[0], i] = -1
        boundary_matrix[edge[1], i] = 1
    return boundary_matrix

def boundary_operator_2(faces, edges):
    boundary_matrix = np.zeros((len(edges), len(faces)))
    for i, face in enumerate(faces):
        for j, edge in enumerate(edges):
            if set(edge).issubset(face):
                if face.index(edge[1]) == (face.index(edge[0]) + 1) % 3:
                    boundary_matrix[j, i] = 1
                else:
                    boundary_matrix[j, i] = -1
    return boundary_matrix

def compute_graph_cohomology(adj_matrix):
    # Extract the size of the matrix to determine vertices
    num_vertices = adj_matrix.shape[0]
    vertices = [(i,) for i in range(num_vertices)]
    
    # Extract edges from the adjacency matrix
    edges = [(i, j) for i in range(num_vertices) for j in range(i) if adj_matrix[i, j] > 0]

    # Compute faces assuming a structure similar to a tetrahedron
    # This computes all possible combinations of 3 vertices, assuming each forms a face
    # This is a simplification and might not be valid for all graph types
    faces = [face for face in combinations(range(num_vertices), 3)]

    print(faces)

    # Compute boundary matrices using predefined functions
    boundary_1 = boundary_operator_1(edges, vertices)
    boundary_2 = boundary_operator_2(faces, edges) if faces else None

    # Calculate cohomology groups
    coboundary_1 = boundary_1.T
    H_cohomology_0 = null_space(coboundary_1).shape[1]  # Number of connected components

    # For H^1, we need to ensure the boundary_1 matrix is defined
    if boundary_1 is not None:
        #H_cohomology_1 = null_space(coboundary_1).shape[1] - np.linalg.matrix_rank(boundary_1)
        H_cohomology_1 = null_space(coboundary_1).shape[1] - np.linalg.matrix_rank(boundary_1)
        print(f'null_space(coboundary_1).shape[1]:  {null_space(coboundary_1).shape[1]}')
        print(f'null_space(boundary_1).shape[1]: {null_space(boundary_1).shape[1]}')

    else:
        H_cohomology_1 = "Not computed - Boundary_1 matrix undefined"

    # For H^2, we need to ensure the boundary_2 matrix is defined and faces are identified
    if boundary_2 is not None and faces:
        coboundary_2 = boundary_2.T
        H_cohomology_2 = len(faces) - np.linalg.matrix_rank(boundary_2)
    else:
        H_cohomology_2 = "Not computed - Boundary_2 matrix undefined or faces not identified"

    return H_cohomology_0, H_cohomology_1, H_cohomology_2

# Example usage with a given adjacency matrix
adj_matrix_example = np.array([
    [0, 1, 1, 1],
    [1, 0, 1, 1],
    [1, 1, 0, 1],
    [1, 1, 1, 0]
])

H_cohomology_groups_example = compute_graph_cohomology(adj_matrix_example)
print("Cohomology Groups for the Example Graph:")
print("H^0:", H_cohomology_groups_example[0], 
      ", H^1:", H_cohomology_groups_example[1], 
      ", H^2:", H_cohomology_groups_example[2])


[(0, 1, 2), (0, 1, 3), (0, 2, 3), (1, 2, 3)]
null_space(coboundary_1).shape[1]:  1
null_space(boundary_1).shape[1]: 3
Cohomology Groups for the Example Graph:
H^0: 1 , H^1: -2 , H^2: 1


In [8]:
import numpy as np
from scipy.linalg import null_space
from itertools import combinations

# Boundary operator for 1-simplices (edges)
def boundary_operator_1(edges, vertices):
    boundary_matrix = np.zeros((len(vertices), len(edges)))
    for i, edge in enumerate(edges):
        boundary_matrix[edge[0], i] = -1
        boundary_matrix[edge[1], i] = 1
    return boundary_matrix

# Boundary operator for 2-simplices (faces)
def boundary_operator_2(faces, edges):
    boundary_matrix = np.zeros((len(edges), len(faces)))
    for i, face in enumerate(faces):
        for j, edge in enumerate(edges):
            if set(edge).issubset(face):
                if (face.index(edge[1]) - face.index(edge[0])) % 3 == 1:
                    boundary_matrix[j, i] = 1
                else:
                    boundary_matrix[j, i] = -1
    return boundary_matrix

# Compute graph cohomology groups
def compute_graph_cohomology(adj_matrix):
    num_vertices = adj_matrix.shape[0]
    vertices = list(range(num_vertices))
    edges = [(i, j) for i in range(num_vertices) for j in range(i) if adj_matrix[i, j] > 0]
    faces = [face for face in combinations(vertices, 3)]

    print(faces)

    boundary_1 = boundary_operator_1(edges, vertices)
    boundary_2 = boundary_operator_2(faces, edges)

    coboundary_1 = boundary_1.T
    H_cohomology_0 = null_space(coboundary_1).shape[1]

    # H^1 as the dimension of the kernel of δ^1
    H_cohomology_1 = null_space(coboundary_1).shape[1]

    # H^2 as the dimension of the kernel of δ^2
    coboundary_2 = boundary_2.T
    H_cohomology_2 = null_space(coboundary_2).shape[1] - np.linalg.matrix_rank(boundary_2)

    return H_cohomology_0, H_cohomology_1, H_cohomology_2

# Adjacency matrix for a fully connected graph of four vertices
adj_matrix_example = np.array([
    [0, 1, 0, 0, 1, 1],
    [1, 0, 1, 0, 0, 1],
    [0, 1, 0, 1, 0, 1],
    [0, 0, 1, 0, 1, 1],
    [1, 0, 0, 1, 0, 1],
    [1, 1, 1, 1, 1, 1]
])

# Compute and print the cohomology groups for the example graph
H_cohomology_groups_example = compute_graph_cohomology(adj_matrix_example)
H_cohomology_groups_example


[(0, 1, 2), (0, 1, 3), (0, 1, 4), (0, 1, 5), (0, 2, 3), (0, 2, 4), (0, 2, 5), (0, 3, 4), (0, 3, 5), (0, 4, 5), (1, 2, 3), (1, 2, 4), (1, 2, 5), (1, 3, 4), (1, 3, 5), (1, 4, 5), (2, 3, 4), (2, 3, 5), (2, 4, 5), (3, 4, 5)]


(1, 1, -8)

In [9]:
def find_triangular_faces(adj_matrix):
    num_vertices = adj_matrix.shape[0]
    triangular_faces = []

    # Check every combination of three vertices to see if they form a face
    for i in range(num_vertices):
        for j in range(i):
            for k in range(j):
                # If there is an edge between each pair of vertices, we have a triangular face
                if adj_matrix[i][j] and adj_matrix[j][k] and adj_matrix[k][i]:
                    triangular_faces.append((i, j, k))

    # Now we need to ensure that each edge is part of at most two faces
    edge_in_faces = {}

    for face in triangular_faces:
        edges = [(face[i], face[(i+1) % 3]) for i in range(3)] # Get edges of the triangle
        for edge in edges:
            edge = tuple(sorted(edge))  # Sort the edge tuple for consistency
            if edge not in edge_in_faces:
                edge_in_faces[edge] = 0
            edge_in_faces[edge] += 1

            # If an edge is in more than 2 faces, remove the current face
            if edge_in_faces[edge] > 2:
                for e in edges:
                    e = tuple(sorted(e))
                    edge_in_faces[e] -= 1
                triangular_faces.remove(face)
                break

    return triangular_faces


In [10]:
find_triangular_faces(adj_matrix_example)

[(5, 1, 0), (5, 2, 1), (5, 3, 2), (5, 4, 0), (5, 4, 3)]

In [25]:
import numpy as np
from scipy.linalg import null_space
from itertools import combinations

def find_triangular_faces(adj_matrix):
    num_vertices = adj_matrix.shape[0]
    triangular_faces = []

    # Check every combination of three vertices to see if they form a face
    for i in range(num_vertices):
        for j in range(i):
            for k in range(j):
                # If there is an edge between each pair of vertices, we have a triangular face
                if adj_matrix[i][j] and adj_matrix[j][k] and adj_matrix[k][i]:
                    triangular_faces.append((i, j, k))

    # Now we need to ensure that each edge is part of at most two faces
    edge_in_faces = {}

    for face in triangular_faces:
        edges = [(face[i], face[(i+1) % 3]) for i in range(3)] # Get edges of the triangle
        for edge in edges:
            edge = tuple(sorted(edge))  # Sort the edge tuple for consistency
            if edge not in edge_in_faces:
                edge_in_faces[edge] = 0
            edge_in_faces[edge] += 1

            # If an edge is in more than 2 faces, remove the current face
            if edge_in_faces[edge] > 2:
                for e in edges:
                    e = tuple(sorted(e))
                    edge_in_faces[e] -= 1
                triangular_faces.remove(face)
                break

    return triangular_faces


# Boundary operator for 1-simplices (edges)
def boundary_operator_1(edges, vertices):
    boundary_matrix = np.zeros((len(vertices), len(edges)))
    for i, edge in enumerate(edges):
        boundary_matrix[edge[0], i] = -1
        boundary_matrix[edge[1], i] = 1
    return boundary_matrix

# Boundary operator for 2-simplices (faces)
def boundary_operator_2(faces, edges):
    boundary_matrix = np.zeros((len(edges), len(faces)))
    for i, face in enumerate(faces):
        for j, edge in enumerate(edges):
            if set(edge).issubset(face):
                if (face.index(edge[1]) - face.index(edge[0])) % 3 == 1:
                    boundary_matrix[j, i] = 1
                else:
                    boundary_matrix[j, i] = -1
    return boundary_matrix

# Compute graph cohomology groups
def compute_graph_cohomology(adj_matrix):
    num_vertices = adj_matrix.shape[0]
    vertices = list(range(num_vertices))
    edges = [(i, j) for i in range(num_vertices) for j in range(i) if adj_matrix[i, j] > 0]
    
    # Use the provided find_triangular_faces function to find the faces
    faces = find_triangular_faces(adj_matrix)

    print(vertices)
    print(edges)

    # Debugging print statements - remove or comment out in production
    print("Faces:", faces)

    boundary_1 = boundary_operator_1(edges, vertices)
    boundary_2 = boundary_operator_2(faces, edges) if faces else None

    coboundary_1 = boundary_1.T
    H_cohomology_0 = null_space(coboundary_1).shape[1]

    # H^1 as the dimension of the kernel of δ^1
    H_cohomology_1 = null_space(coboundary_1).shape[1]

    # H^2 as the dimension of the kernel of δ^2
    H_cohomology_2 = 'Not computed'  # Default value
    if boundary_2 is not None:
        coboundary_2 = boundary_2.T

        H_cohomology_2 = null_space(coboundary_2).shape[1] - np.linalg.matrix_rank(boundary_2)

    return H_cohomology_0, H_cohomology_1, H_cohomology_2

In [ ]:

# Adjacency matrix for a specific graph (for example, a 6-wheel graph)
adj_matrix_example = np.array([
    [0, 1, 0, 0, 1, 1],
    [1, 0, 1, 0, 0, 1],
    [0, 1, 0, 1, 0, 1],
    [0, 0, 1, 0, 1, 1],
    [1, 0, 0, 1, 0, 1],
    [1, 1, 1, 1, 1, 0]
])

# Compute and print the cohomology groups for the example graph
H_cohomology_groups_example = compute_graph_cohomology(adj_matrix_example)
H_cohomology_groups_example


In [32]:
adj2=np.array([
    [0,1,1,0,0,0],
    [1,0,1,1,1,0],
    [1,1,0,1,1,1],
    [0,1,1,0,1,0],
    [0,1,1,1,0,1],
    [0,0,1,0,1,0]
])

print("The triangles are: ", find_triangles(adj2))

The triangles are:  [[0, 1, 2], [1, 2, 3], [1, 2, 4], [1, 3, 4], [2, 3, 4], [2, 4, 5]]


In [31]:
def find_triangles(adj, n=None):
    if n is None:
        n = len(adj)
    
    triangles = []
    for i in range(n):
        for j in range(i + 1, n):
            for k in range(j + 1, n):
                if (adj[i][j] and adj[j][k] and adj[k][i]):
                    triangles.append([i, j, k])
    return triangles

print("The triangles are: ", find_triangles(adj2))
## >> The triangles are:  [[1, 2, 3]]

The triangles are:  [[0, 1, 2], [1, 2, 3], [1, 2, 4], [1, 3, 4], [2, 3, 4], [2, 4, 5]]


In [28]:
find_triangular_faces(adj2)

KeyError: (1, 4)

In [27]:
compute_graph_cohomology(adj2)

KeyError: (1, 4)

In [36]:
import networkx as nx

# Create a graph (replace this with your actual graph)
G = nx.Graph()
# Add edges to the graph (replace these with your actual edges)
G.add_edges_from(edges2)

# Check if the graph is planar
is_planar, embedding = nx.check_planarity(G)

if is_planar:
    # Find all cliques in the graph
    cliques = list(nx.enumerate_all_cliques(G))
    # Filter cliques to find all 3-cycles, which are cliques of size 3
    triangles = [clique for clique in cliques if len(clique) == 3]
    print("The 3-cycles (triangles) in the graph are:", triangles)
else:
    print("The graph is not planar.")


The 3-cycles (triangles) in the graph are: [[1, 0, 2], [1, 2, 3], [1, 2, 4], [1, 3, 4], [2, 3, 4], [2, 4, 5]]


In [34]:
edges2 = [(i, j) for i in range(6) for j in range(i) if adj2[i, j] > 0]

In [35]:
edges2

[(1, 0),
 (2, 0),
 (2, 1),
 (3, 1),
 (3, 2),
 (4, 1),
 (4, 2),
 (4, 3),
 (5, 2),
 (5, 4)]